In [46]:
test = """2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5"""
source = test

## Ray's simple test
Six blocks oriented around a hollow center for simpler testing. Can reason about it: total SA 36, exterior SA 30

In [47]:
test="""2,2,1
1,2,2
2,1,2
3,2,2
2,3,2
2,2,3"""
# source = test

In [48]:
source = open("./day_18.txt").read()

## Part 1: Surface area

In [49]:
sides = {}
for dims in [list(map(int, line.split(","))) for line in source.splitlines()]:
    strs = [str(d) for d in dims]
    d = ",".join(strs)
    key = "x" + d
    sides[key] =   sides.get(key,0)+1
    key = "y" + d
    sides[key] =   sides.get(key,0)+1
    key = "z" + d
    sides[key] =   sides.get(key,0)+1
    strx = [str(dims[0]+1), strs[1], strs[2]]
    key = "x" + ",".join(strx)
    sides[key] =   sides.get(key,0)-1
    stry = [strs[0], str(dims[1]+1), strs[2]]
    key = "y" + ",".join(stry)
    sides[key] =   sides.get(key,0)-1
    strz = [strs[0], strs[1], str(dims[2]+1)]
    key = "z" + ",".join(strz)
    sides[key] =   sides.get(key,0)-1

goodSides =([(s, sides[s]) for s in sides.keys() if abs(sides[s]) == 1])
print(len(goodSides))

4512


## Part 2: Outside only
Strategy: Make a "mold" of a grid of elements that are filled outside of the field. Just walk from the corner I know is open with one block of space on each side of the interior object. The resulting object should have a total surface area of the SA of the lava molecule + the exterior surface area of the prism. Since calculating the latter is trivial, the difference is my answer! Let's see if it works...

In [50]:
minmax = [[min(d), max(d)] for d in zip(*[list(map(int, line.split(","))) for line in source.splitlines()])]

In [51]:
import numpy
bounds = [(mm[0]-1, mm[1]+1) for mm in minmax]

field = numpy.zeros((bounds[0][1]-bounds[0][0]+1, bounds[1][1]-bounds[1][0]+1, bounds[2][1]-bounds[2][0]+1, 1), dtype=bool)
field2 = numpy.zeros((bounds[0][1]-bounds[0][0]+1, bounds[1][1]-bounds[1][0]+1, bounds[2][1]-bounds[2][0]+1, 1), dtype=int)

for xyz in [[int(c) for c in line.split(",")] for line in source.splitlines()]:
    newCoords = numpy.diff(list(zip([numpy.array(bounds)[:,0], numpy.array(xyz)])), axis = 0)[0][0]
    field[newCoords[0], newCoords[1], newCoords[2]] = True

In [52]:
zerozerozero = numpy.array([0,0,0])
pointsToCheck = [zerozerozero]

def check_point(point):
    if(point[0] < 0 or point[1] < 0 or point[2] < 0): return False
    if(point[0] >= len(field) or point[1] >= len(field[0]) or point[2] >= len(field[0][0])): return False
    return field2[point[0], point[1], point[2]] == 0 and field[point[0], point[1], point[2]] == False 

while(len(pointsToCheck) > 0):
    point = pointsToCheck.pop()
    if(check_point(point)):
        field2[point[0], point[1], point[2]] = 1
        pointsToCheck.extend([point + numpy.array([0,0,1]), point + numpy.array([0,0,-1]), point + numpy.array([0,1,0]), point + numpy.array([0,-1,0]), point + numpy.array([1,0,0]), point + numpy.array([-1,0,0])])


In [53]:
sides = {}
for dims in [(x,y,z) for x in range(len(field2)) for y in range(len(field2[0])) for z in range(len(field2[0][0]))]:
    if(field2[dims[0], dims[1], dims[2]] == 1):
        strs = [str(d) for d in dims]
        d = ",".join(strs)
        key = "x" + d
        sides[key] =   sides.get(key,0)+1
        key = "y" + d
        sides[key] =   sides.get(key,0)+1
        key = "z" + d
        sides[key] =   sides.get(key,0)+1
        strx = [str(dims[0]+1), strs[1], strs[2]]
        key = "x" + ",".join(strx)
        sides[key] =   sides.get(key,0)-1
        stry = [strs[0], str(dims[1]+1), strs[2]]
        key = "y" + ",".join(stry)
        sides[key] =   sides.get(key,0)-1
        strz = [strs[0], strs[1], str(dims[2]+1)]
        key = "z" + ",".join(strz)
        sides[key] =   sides.get(key,0)-1
goodSides =([(s, sides[s]) for s in sides.keys() if abs(sides[s]) == 1])
exteriorSA = len(field2)* len(field2[0]) * 2 + len(field2)* len(field2[0][0]) * 2 + len(field2[0])* len(field2[0][0]) * 2
print(len(goodSides)- exteriorSA)


2554
